In [12]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from keras.models import load_model
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM
from keras.layers import Dropout
from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error, mean_squared_error

In [6]:
data = pd.read_csv('./data/iots.txt', delimiter = "\t")
data = data.iloc[:, :10]
combined_data = data
combined_data.head()

,Unnamed: 0,308511_speed,308511_flow,308512_speed,308512_flow,311903_speed,311903_flow,311974_speed,311974_flow,312010_speed
0,05/23/2024 00:00:00,66.1,17.0,63.8,43.0,66.5,99.0,70.7,108.0,70.7
1,05/23/2024 00:05:00,66.0,19.0,63.7,45.0,66.5,84.0,69.7,103.0,69.7
2,05/23/2024 00:10:00,65.9,19.0,63.7,43.0,66.4,92.0,69.4,106.0,69.4
3,05/23/2024 00:15:00,66.1,15.0,63.6,42.0,66.5,82.0,70.0,100.0,70.0
4,05/23/2024 00:20:00,66.0,17.0,63.7,41.0,66.4,96.0,69.7,98.0,69.7


In [7]:
combined_data.rename(columns={'Unnamed: 0': 'time'}, inplace=True)
# Set the 'time' column as the index
combined_data.set_index('time', inplace=True)

# Convert the index into a proper datetime format
combined_data.index = pd.to_datetime(data.index)

# Check the first few rows to confirm
combined_data.head()

,308511_speed,308511_flow,308512_speed,308512_flow,311903_speed,311903_flow,311974_speed,311974_flow,312010_speed
time,,,,,,,,,
2024-05-23 00:00:00,66.1,17.0,63.8,43.0,66.5,99.0,70.7,108.0,70.7
2024-05-23 00:05:00,66.0,19.0,63.7,45.0,66.5,84.0,69.7,103.0,69.7
2024-05-23 00:10:00,65.9,19.0,63.7,43.0,66.4,92.0,69.4,106.0,69.4
2024-05-23 00:15:00,66.1,15.0,63.6,42.0,66.5,82.0,70.0,100.0,70.0
2024-05-23 00:20:00,66.0,17.0,63.7,41.0,66.4,96.0,69.7,98.0,69.7


In [13]:
def create_sequences(data, seq_length):
    X, y = [], []

    for i in range(len(data) - seq_length):
        seq = data[i:(i + seq_length)]
        X.append(seq[:-1])
        y.append(seq[-1])

    return np.array(X), np.array(y)

data_308511 = combined_data[['308511_speed', '308511_flow']]
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_308511)

# Choose a sequence length (e.g., 10 time steps)
seq_length = 10
X, y = create_sequences(scaled_data, seq_length)
train_size = int(len(X) * 0.67)  # Taking 67% of data for training
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]
# Reshape input to be [samples, time steps, features]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 2))
X_test = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 2))

In [30]:
X_train.shape, X_test.shape

((186, 9, 2), (92, 9, 2))

In [14]:
# LSTM Model
model = Sequential()
model.add(LSTM(units=50, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])))
model.add(LSTM(units=50))
model.add(Dense(units=2))  # for predicting both speed and flow
model.compile(optimizer='adam', loss='mean_squared_error')

c:\Users\yffan\Documents\VScodeProject\Python\smartcity-backend\.venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


In [15]:
history = model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test), verbose=1)

Epoch 1/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 0.3299 - val_loss: 0.1382
Epoch 2/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.1360 - val_loss: 0.0176
Epoch 3/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step - loss: 0.0536 - val_loss: 0.0346
Epoch 4/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0466 - val_loss: 0.0131
Epoch 5/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0304 - val_loss: 0.0146
Epoch 6/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0264 - val_loss: 0.0101
Epoch 7/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0154 - val_loss: 0.0099
Epoch 8/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0165 - val_loss: 0.0104
Epoch 9/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step - loss: 0.0160 - val_loss: 0.0098
Epoch 10/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0159 - val_loss: 0.0093
Epoch 11/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0158 - val_loss: 0.0098
Epoch 12/20
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 9ms/step - loss: 0.0163 - val_loss: 0.0091
Epoch 13/20

In [16]:
y_pred = model.predict(X_test)

3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 95ms/step


In [17]:
# Inverse scale the predictions
y_pred_actual = scaler.inverse_transform(y_pred)
y_test_actual = scaler.inverse_transform(y_test)
# Calculate MAE and RMSE for speed
mae_speed = mean_absolute_error(y_test_actual[:, 0], y_pred_actual[:, 0])
rmse_speed = np.sqrt(mean_squared_error(y_test_actual[:, 0], y_pred_actual[:, 0]))

# Calculate MAE and RMSE for flow
mae_flow = mean_absolute_error(y_test_actual[:, 1], y_pred_actual[:, 1])
rmse_flow = np.sqrt(mean_squared_error(y_test_actual[:, 1], y_pred_actual[:, 1]))

print(f"MAE for Speed: {mae_speed}")
print(f"RMSE for Speed: {rmse_speed}")
print(f"MAE for Flow: {mae_flow}")
print(f"RMSE for Flow: {rmse_flow}")

MAE for Speed: 0.2806625532067347
RMSE for Speed: 0.36029162483329114
MAE for Flow: 4.967082811438519
RMSE for Flow: 6.278737417840075


In [19]:
model.save('./model/iot_model.keras')

In [20]:
model = load_model('./model/iot_model.keras')

c:\Users\yffan\Documents\VScodeProject\Python\smartcity-backend\.venv\Lib\site-packages\keras\src\saving\saving_lib.py:415: UserWarning: Skipping variable loading for optimizer 'rmsprop', because it has 10 variables whereas the saved optimizer has 18 variables. 
  saveable.load_own_variables(weights_store.get(inner_path))


In [31]:
y_pred = model.predict(X)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 


In [26]:
# Inverse scale the predictions
y_pred_actual = scaler.inverse_transform(y_pred)
y_test_actual = scaler.inverse_transform(y_test)
# Calculate MAE and RMSE for speed
mae_speed = mean_absolute_error(y_test_actual[:, 0], y_pred_actual[:, 0])
rmse_speed = np.sqrt(mean_squared_error(y_test_actual[:, 0], y_pred_actual[:, 0]))

# Calculate MAE and RMSE for flow
mae_flow = mean_absolute_error(y_test_actual[:, 1], y_pred_actual[:, 1])
rmse_flow = np.sqrt(mean_squared_error(y_test_actual[:, 1], y_pred_actual[:, 1]))

print(f"MAE for Speed: {mae_speed}")
print(f"RMSE for Speed: {rmse_speed}")
print(f"MAE for Flow: {mae_flow}")
print(f"RMSE for Flow: {rmse_flow}")

MAE for Speed: 0.2806625532067347
RMSE for Speed: 0.36029162483329114
MAE for Flow: 4.967082811438519
RMSE for Flow: 6.278737417840075


In [32]:
y = scaler.inverse_transform(y_pred)
print(y.shape)


(278, 2)


In [39]:
data_308512 = combined_data[['308512_speed', '308512_flow']]
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data_308512)
X, y = create_sequences(scaled_data, seq_length)
X = np.reshape(X, (X.shape[0], X.shape[1], 2))
y_pred = model.predict(X)
y_pred_actual = scaler.inverse_transform(y_pred)
print(y_pred_actual)

9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step 
[[ 63.789177  47.373665]
 [ 63.78096   47.060566]
 [ 63.780704  46.517513]
 [ 63.778744  46.142227]
 [ 63.784397  45.833786]
 [ 63.785263  45.62602 ]
 [ 63.786476  45.43927 ]
 [ 63.78614   45.393784]
 [ 63.791664  45.47935 ]
 [ 63.79355   45.361023]
 [ 63.786453  45.22217 ]
 [ 63.7793    45.047894]
 [ 63.77674   45.181347]
 [ 63.765965  45.253754]
 [ 63.763012  45.5717  ]
 [ 63.76744   45.610844]
 [ 63.777084  45.281837]
 [ 63.79073   45.656963]
 [ 63.804058  45.532375]
 [ 63.81423   45.453114]
 [ 63.815166  45.79436 ]
 [ 63.813885  45.898045]
 [ 63.81024   45.776115]
 [ 63.793484  45.790306]
 [ 63.776016  46.044994]
 [ 63.761417  46.19294 ]
 [ 63.759243  46.077305]
 [ 63.759243  46.068512]
 [ 63.760853  45.88943 ]
 [ 63.765938  45.312843]
 [ 63.763992  45.339577]
 [ 63.762936  45.300907]
 [ 63.7637    45.517105]
 [ 63.765385  45.519623]
 [ 63.770027  46.011547]
 [ 63.77147   46.450874]
 [ 63.765358  46.952908]
 [ 63.76661   47.3418  ]
 [ 63.779427